# Notebook for building an XGBoost model for the Sendgrid sign up dataset
***
**Jake Mitchell Scott Schubert**

Initially using ______

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from numpy import loadtxt
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from random import randint
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from ipywidgets import FloatProgress
from IPython.display import display

In [2]:
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    forwardsma = np.convolve(values, weights, 'valid')
    #perform a moving average backwards, then reverse it back
    backwardsma = np.convolve(values[::-1], weights, 'valid')[::-1]
    

    dualDirectionMA = np.zeros(len(values))
    dualDirectionMA[len(dualDirectionMA)-window+1:] = forwardsma[len(forwardsma)-window+1:]
    dualDirectionMA[:window-1] = backwardsma[:window-1]
    dualDirectionMA[window-1:len(dualDirectionMA)-window+1] = [(a+b)/2 for a,b in zip(forwardsma[:len(forwardsma)-window+1], backwardsma[window-1:])]
    return dualDirectionMA

In [3]:
train_path = 'data/signup_train_data.csv'
test_path = 'data/signup_test_data.csv'
pd.options.display.max_columns = 2000


# Load the data into a DataFrame 
df = pd.read_csv(train_path, low_memory=False)
test_df = pd.read_csv(test_path, low_memory=False)

# Split into X and Y
dfLabels = df.pop("label")
#print (df.head())

#Feature Engineering
#for index, row in df.iterrows():
#    df['time_of_day_int'] = introw['created_at'][11:13])*3600 + int(row['created_at'][14:16])*60 + int(row['created_at'][17:19])
for dataframe in [df, test_df]:
    # df['employee_count_low'] = 0
    dataframe['employee_count_high'] = 0
    # dataframe.loc[df['employee_count']  == '1 - 500', 'employee_count_low'] = 1
    dataframe.loc[df['employee_count']  == '1 - 500', 'employee_count_high'] = 500
    # dataframe.loc[df['employee_count']  == '501 - 1,000', 'employee_count_low'] = 501
    dataframe.loc[df['employee_count']  == '501 - 1,000', 'employee_count_high'] = 1000
    # dataframe.loc[df['employee_count']  == '1,001 - 5,000', 'employee_count_low'] = 1001
    dataframe.loc[df['employee_count']  == '1,001 - 5,000', 'employee_count_high'] = 5000
    # dataframe.loc[df['employee_count']  == '5,000+', 'employee_count_low'] = 5001
    dataframe.loc[df['employee_count']  == '5,000+', 'employee_count_high'] = 10000
    dataframe['volume_high'] = 0
    dataframe.loc[df['volume']  == '1 - 100,000', 'volume_high'] = 100000
    dataframe.loc[dataframe['volume']  == '100,001 - 2,500,000', 'volume_high'] = 2500000
    dataframe.loc[dataframe['volume']  == '2,500,001 - 10,000,000', 'volume_high'] = 10000000
    dataframe.loc[dataframe['volume']  == '10,000,000+', 'volume_high'] = 20000000
    dataframe.loc[dataframe['volume']  == '0 to 40k', 'volume_high'] = 40000
    dataframe.loc[dataframe['volume']  == '40k to 100k', 'volume_high'] = 100000
    dataframe['developer'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Developer', 'developer'] = 1
    dataframe['ceo'] = 0
    dataframe.loc[dataframe['user_persona']  == 'CEO', 'ceo'] = 1
    dataframe['other'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Other', 'other'] = 1
    dataframe['marketing'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Marketing', 'marketing'] = 1
    dataframe['technical_support'] = 0
    dataframe.loc[dataframe['user_persona']  == 'Technical Support', 'technical_support'] = 1



#Temporarily parse non numerical data
#,'user_persona'
frames_to_remove = ['zip', 'city', 'website', 'state', 'country', 'registration_ip', 'company','multifactor_country_code','created_at','lead_source','marketing_channel','initial_package','geolocation_notes','name_notes','ip_notes','community_notes','email_notes','activity_notes','fingerprint_notes','employee_count','volume','user_persona']
df = df.drop(frames_to_remove, axis=1)
test_df = test_df.drop(frames_to_remove, axis=1)

df.fillna(value=0, inplace=True)
test_df.fillna(value=0, inplace=True)
df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

df.head()




,index,id,banned_ip,ip_count,is_authy_verified,banned_email,is_transactional,is_marketing,is_behavioral,is_oem,risk,geolocation_risk,name_risk,ip_risk,community_risk,fingerprint_risk,email_risk,activity_risk,ip_cluster,email_cluster,mfa_required,mfa_completed,whitelabel_required,whitelabel_completed,payment_required,payment_completed,profile_completed,email_completed,employee_count_high,volume_high,developer,ceo,other,marketing,technical_support
0,0,0,6,8,0,1,0,0,0,0,-100,0,0,-155,-119,-10,-8,0,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,500,2500000,1,0,0,0,0
1,1,1,5,5,0,1,1,0,0,0,-90,0,0,-25,-37,-10,-8,0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,100000,1,0,0,0,0
2,2,2,0,1,0,0,1,0,0,0,-44,-16,0,-10,0,0,-8,0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,100000,0,1,0,0,0
3,3,3,4,4,0,2,0,1,0,0,-72,-14,0,-40,0,0,-8,0,False,False,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,500,100000,1,0,0,0,0
4,4,4,0,1,0,0,1,0,0,0,-34,-16,0,0,0,0,-8,0,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,100000,0,0,1,0,0


In [4]:
print(df['risk'].unique())

[-100  -90  -44  -72  -34  -19   -9  -18  -54  -52  -16  -51  -41  -28
  -37  -46  -33  -29  -15  -38  -56  -17  -66  -31  -25  -61  -57  -32
  -98  -77  -47  -94  -62  -23  -21  -24   -8  -42  -91  -14  -60  -35
  -71  -45  -20  -96  -81  -53  -43  -65  -93  -49  -64  -36  -10  -99
  -67  -95  -85  -80  -75  -74  -73  -68  -70  -79  -12  -69  -55  -88
  -39  -59  -11  -89  -48  -58  -63  -30  -82  -50  -27  -78  -76  -87
  -84  -22  -92  -86  -83  -97  -40  -26  100   95   -7  -13    1   78
   80   94   85   75    0   -3   74   55   -4   -5   -6   -1   -2   64
   72   88   92   81   86   84   61   98   62   70   65   99   48   69
   59   56   96   68   44   83]


In [21]:
def CrossValidateClassifier(classifierFunction, standardize=False, **kwargs):
    '''The classifierFunction should be a function that takes (x_train, y_train, X_valid, Y_valid, **kwargs)
       and returns (train_accuracy, valid_accuracy) from that classifier'''
    splits = 2
    accuracys = np.zeros((2,splits), dtype=float)
    
    print("Running Cross Validation on {0} with parameters {1}".format(classifierFunction.__name__, kwargs))
    skf = StratifiedKFold(n_splits=splits, shuffle=True)
    for i, (train_indexes, valid_indexes) in enumerate(skf.split(df, dfLabels)):
        fold_X_train, fold_y_train = df.iloc[train_indexes], dfLabels.iloc[train_indexes]  
        fold_X_valid, fold_y_valid = df.iloc[valid_indexes], dfLabels.iloc[valid_indexes]    

        (accuracys[0][i], accuracys[1][i]) = classifierFunction(fold_X_train, fold_y_train, fold_X_valid, fold_y_valid, kwargs)
        print("    Fold {1}: Train Accuracy: {2:.4f} Valid Accuracy: {3:.4f}".format(kwargs, i, accuracys[0][i] , accuracys[1][i]))
    meanAccuracy = (np.mean(accuracys[0]), np.mean(accuracys[1]))
    print("{0} with parameters {1} === Train Accuracy: {2:.4f} Mean Valid Accuracy: {3:.4f}".format(classifierFunction.__name__, kwargs, meanAccuracy[0], meanAccuracy[1]))
    print("\n\n")
    return meanAccuracy

In [29]:
def FinalAccuracyClassifier(classifierModel, **kwargs):
    '''The classifierFunction should be a function that takes (x_train, y_train, **kwargs)
       trains a model and returns final accuracy from that classifier'''
    model = classifierModel(df,dfLabels,kwargs)
    y_pred = model.predict(test_df)
    predictions = [round(value) for value in y_pred]

    d = {'id':test_df['id'], 'label':predictions}
    preds = pd.DataFrame(data=d)

    preds.to_csv('predictions.csv', index=False)

In [ ]:
def LSVMClassifier(X_train, y_train, X_valid, y_valid, hyperparameters):
    '''Takes (x_train, y_train, X_valid, Y_valid, {})
       returns (train_accuracy, valid_accuracy)
       
       We want to use the dual false (run the primal problem) due to the size of the dataset'''
    lsvm = LinearSVC(dual=hyperparameters["dual"])
    lsvm.fit(X_train,y_train)
    return (lsvm.score(X_train, y_train), lsvm.score(X_valid,y_valid))

In [ ]:
CrossValidateClassifier(LSVMClassifier, dual=True)

In [7]:
def AdaClassifier(X_train, y_train, X_valid, y_valid, hyperparameters):
    '''Takes (x_train, y_train, X_valid, Y_valid, {})
       returns (train_accuracy, valid_accuracy)'''
    ada = AdaBoostClassifier(base_estimator=hyperparameters['base_estimator'], n_estimators=hyperparameters['n_estimators'], learning_rate=hyperparameters['learning_rate'])
    ada.fit(X_train,y_train)
    return (ada.score(X_train, y_train), ada.score(X_valid,y_valid))

In [23]:
def AdaPredClassifier(X_train, y_train, hyperparameters):
    ada = AdaBoostClassifier(base_estimator=hyperparameters['base_estimator'], n_estimators=hyperparameters['n_estimators'], learning_rate=hyperparameters['learning_rate'])
    ada.fit(X_train,y_train)
    return ada

In [14]:
f = FloatProgress(min=0, max=3*5*4)
display(f)

FloatProgress(value=0.0, max=60.0)

In [33]:
'''Apparently there is a trade off between n and learning rate, also taking a look at depth because of how important it is'''
adaAccuracyList = np.zeros((5,1), dtype=float)
i = 0
for n in [50]:
    for learning_rate in [0.9]:
        for depth in [1]:
            adaAccuracyList[0][i] = n
            adaAccuracyList[1][i] = learning_rate
            adaAccuracyList[2][i] = depth
            (adaAccuracyList[3][i], adaAccuracyList[4][i]) = CrossValidateClassifier(AdaClassifier, base_estimator=DecisionTreeClassifier(max_depth=depth), n_estimators=n, learning_rate=learning_rate)
            f.value += 1
            i += 1

Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9401 Valid Accuracy: 0.9396
    Fold 1: Train Accuracy: 0.9407 Valid Accuracy: 0.9409
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 

Current best hyper parameters are 150 estimators, 0.9 learning rate, depth of 4.  With this kind of stuff we can just up the depth and estimators and keep getting little benefits

In [34]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4), n_estimators=150, learning_rate=0.9)
ada.fit(df, dfLabels)
print(ada.score(df, dfLabels))
yhat = ada.predict(test_df)
print(yhat)


0.9746470774962117
[0 1 0 ... 0 1 0]


In [36]:
predictions = [round(value) for value in yhat]

d = {'id':test_df['id'], 'label':predictions}
preds = pd.DataFrame(data=d)

preds.to_csv('predictions.csv', index=False)

In [ ]:
def xgb_basic(seed_in, TO_TEST, eta=0.1, depth=3, n_estimators=100, split=0.5):
    # split data into train and test sets
    seed = seed_in
    X_train, X_test, y_train, y_test = train_test_split(df, dfLabels, test_size=1-split, random_state=seed)

    # fit model no training data
    model = XGBClassifier(learning_rate=eta, max_depth=depth, n_estimators=n_estimators)
    model.fit(X_train, y_train)
    
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Seed: %i Eta: %f Depth: %i Estimators: %i  -  Accuracy: %.3f%%" % (seed, eta, depth, n_estimators, accuracy * 100.0))
    
    if TO_TEST:
        #make predictions for train data
        y_pred = model.predict(test_df)
        predictions = [round(value) for value in y_pred]

        d = {'id':test_df['id'], 'label':predictions}
        preds = pd.DataFrame(data=d)

        preds.to_csv('predictions.csv', index=False)

In [ ]:
# for split in [0.5,0.6,0.7,0.75,0.8,0.85,0.9,0.95]:
#     xgb_basic(31,False, depth=9, eta=0.15, n_estimators=n_estimators, split=split)

In [ ]:
xgb_basic(1, False, depth=9, eta=0.15, n_estimators=120)

In [ ]:
xgb_basic(1, True, depth=9, eta=0.15, n_estimators=120, split=0.7)

In [ ]:
Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9380 Valid Accuracy: 0.9389
    Fold 1: Train Accuracy: 0.9390 Valid Accuracy: 0.9386
    Fold 2: Train Accuracy: 0.9418 Valid Accuracy: 0.9410
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9} === Train Accuracy: 0.9396 Mean Valid Accuracy: 0.9395



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9567 Valid Accuracy: 0.9572
    Fold 1: Train Accuracy: 0.9564 Valid Accuracy: 0.9561
    Fold 2: Train Accuracy: 0.9570 Valid Accuracy: 0.9559
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9} === Train Accuracy: 0.9567 Mean Valid Accuracy: 0.9564



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9628 Valid Accuracy: 0.9619
    Fold 1: Train Accuracy: 0.9629 Valid Accuracy: 0.9626
    Fold 2: Train Accuracy: 0.9634 Valid Accuracy: 0.9627
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9} === Train Accuracy: 0.9630 Mean Valid Accuracy: 0.9624



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9676 Valid Accuracy: 0.9658
    Fold 1: Train Accuracy: 0.9677 Valid Accuracy: 0.9670
    Fold 2: Train Accuracy: 0.9663 Valid Accuracy: 0.9643
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.9} === Train Accuracy: 0.9672 Mean Valid Accuracy: 0.9657



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9361 Valid Accuracy: 0.9365
    Fold 1: Train Accuracy: 0.9391 Valid Accuracy: 0.9387
    Fold 2: Train Accuracy: 0.9402 Valid Accuracy: 0.9398
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95} === Train Accuracy: 0.9384 Mean Valid Accuracy: 0.9384



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9568 Valid Accuracy: 0.9561
    Fold 1: Train Accuracy: 0.9566 Valid Accuracy: 0.9571
    Fold 2: Train Accuracy: 0.9557 Valid Accuracy: 0.9550
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95} === Train Accuracy: 0.9564 Mean Valid Accuracy: 0.9561



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9631 Valid Accuracy: 0.9635
    Fold 1: Train Accuracy: 0.9635 Valid Accuracy: 0.9611
    Fold 2: Train Accuracy: 0.9627 Valid Accuracy: 0.9628
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95} === Train Accuracy: 0.9631 Mean Valid Accuracy: 0.9625



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9685 Valid Accuracy: 0.9671
    Fold 1: Train Accuracy: 0.9680 Valid Accuracy: 0.9657
    Fold 2: Train Accuracy: 0.9670 Valid Accuracy: 0.9665
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 0.95} === Train Accuracy: 0.9678 Mean Valid Accuracy: 0.9664



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9415 Valid Accuracy: 0.9410
    Fold 1: Train Accuracy: 0.9367 Valid Accuracy: 0.9359
    Fold 2: Train Accuracy: 0.9380 Valid Accuracy: 0.9391
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1} === Train Accuracy: 0.9387 Mean Valid Accuracy: 0.9386



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9561 Valid Accuracy: 0.9545
    Fold 1: Train Accuracy: 0.9544 Valid Accuracy: 0.9551
    Fold 2: Train Accuracy: 0.9572 Valid Accuracy: 0.9573
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1} === Train Accuracy: 0.9559 Mean Valid Accuracy: 0.9556



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9650 Valid Accuracy: 0.9644
    Fold 1: Train Accuracy: 0.9639 Valid Accuracy: 0.9635
    Fold 2: Train Accuracy: 0.9649 Valid Accuracy: 0.9645
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1} === Train Accuracy: 0.9646 Mean Valid Accuracy: 0.9641



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9681 Valid Accuracy: 0.9668
    Fold 1: Train Accuracy: 0.9687 Valid Accuracy: 0.9671
    Fold 2: Train Accuracy: 0.9678 Valid Accuracy: 0.9662
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1} === Train Accuracy: 0.9682 Mean Valid Accuracy: 0.9667



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9374 Valid Accuracy: 0.9369
    Fold 1: Train Accuracy: 0.9398 Valid Accuracy: 0.9417
    Fold 2: Train Accuracy: 0.9387 Valid Accuracy: 0.9378
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05} === Train Accuracy: 0.9386 Mean Valid Accuracy: 0.9388



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9567 Valid Accuracy: 0.9567
    Fold 1: Train Accuracy: 0.9556 Valid Accuracy: 0.9570
    Fold 2: Train Accuracy: 0.9562 Valid Accuracy: 0.9545
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05} === Train Accuracy: 0.9561 Mean Valid Accuracy: 0.9561



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9626 Valid Accuracy: 0.9624
    Fold 1: Train Accuracy: 0.9621 Valid Accuracy: 0.9613
    Fold 2: Train Accuracy: 0.9629 Valid Accuracy: 0.9624
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05} === Train Accuracy: 0.9625 Mean Valid Accuracy: 0.9620



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9677 Valid Accuracy: 0.9666
    Fold 1: Train Accuracy: 0.9671 Valid Accuracy: 0.9652
    Fold 2: Train Accuracy: 0.9684 Valid Accuracy: 0.9664
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.05} === Train Accuracy: 0.9677 Mean Valid Accuracy: 0.9661



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9397 Valid Accuracy: 0.9394
    Fold 1: Train Accuracy: 0.9397 Valid Accuracy: 0.9387
    Fold 2: Train Accuracy: 0.9398 Valid Accuracy: 0.9417
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1} === Train Accuracy: 0.9398 Mean Valid Accuracy: 0.9399



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9545 Valid Accuracy: 0.9539
    Fold 1: Train Accuracy: 0.9549 Valid Accuracy: 0.9551
    Fold 2: Train Accuracy: 0.9560 Valid Accuracy: 0.9552
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1} === Train Accuracy: 0.9551 Mean Valid Accuracy: 0.9547



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9635 Valid Accuracy: 0.9616
    Fold 1: Train Accuracy: 0.9623 Valid Accuracy: 0.9609
    Fold 2: Train Accuracy: 0.9640 Valid Accuracy: 0.9643
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1} === Train Accuracy: 0.9633 Mean Valid Accuracy: 0.9623



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9677 Valid Accuracy: 0.9663
    Fold 1: Train Accuracy: 0.9679 Valid Accuracy: 0.9660
    Fold 2: Train Accuracy: 0.9674 Valid Accuracy: 0.9660
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 30, 'learning_rate': 1.1} === Train Accuracy: 0.9677 Mean Valid Accuracy: 0.9661



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9400 Valid Accuracy: 0.9410
    Fold 1: Train Accuracy: 0.9400 Valid Accuracy: 0.9377
    Fold 2: Train Accuracy: 0.9403 Valid Accuracy: 0.9412
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9} === Train Accuracy: 0.9401 Mean Valid Accuracy: 0.9400



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9597 Valid Accuracy: 0.9603
    Fold 1: Train Accuracy: 0.9603 Valid Accuracy: 0.9594
    Fold 2: Train Accuracy: 0.9593 Valid Accuracy: 0.9586
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9} === Train Accuracy: 0.9598 Mean Valid Accuracy: 0.9594



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9653 Valid Accuracy: 0.9638
    Fold 1: Train Accuracy: 0.9660 Valid Accuracy: 0.9652
    Fold 2: Train Accuracy: 0.9653 Valid Accuracy: 0.9645
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9} === Train Accuracy: 0.9655 Mean Valid Accuracy: 0.9645



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9706 Valid Accuracy: 0.9679
    Fold 1: Train Accuracy: 0.9700 Valid Accuracy: 0.9669
    Fold 2: Train Accuracy: 0.9705 Valid Accuracy: 0.9680
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.9} === Train Accuracy: 0.9703 Mean Valid Accuracy: 0.9676



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9403 Valid Accuracy: 0.9404
    Fold 1: Train Accuracy: 0.9410 Valid Accuracy: 0.9415
    Fold 2: Train Accuracy: 0.9420 Valid Accuracy: 0.9409
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95} === Train Accuracy: 0.9411 Mean Valid Accuracy: 0.9409



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9578 Valid Accuracy: 0.9575
    Fold 1: Train Accuracy: 0.9601 Valid Accuracy: 0.9603
    Fold 2: Train Accuracy: 0.9596 Valid Accuracy: 0.9596
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95} === Train Accuracy: 0.9592 Mean Valid Accuracy: 0.9591



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9649 Valid Accuracy: 0.9635
    Fold 1: Train Accuracy: 0.9650 Valid Accuracy: 0.9633
    Fold 2: Train Accuracy: 0.9644 Valid Accuracy: 0.9647
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95} === Train Accuracy: 0.9647 Mean Valid Accuracy: 0.9638



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9702 Valid Accuracy: 0.9681
    Fold 1: Train Accuracy: 0.9702 Valid Accuracy: 0.9682
    Fold 2: Train Accuracy: 0.9702 Valid Accuracy: 0.9670
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 0.95} === Train Accuracy: 0.9702 Mean Valid Accuracy: 0.9678



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9411 Valid Accuracy: 0.9407
    Fold 1: Train Accuracy: 0.9403 Valid Accuracy: 0.9410
    Fold 2: Train Accuracy: 0.9414 Valid Accuracy: 0.9410
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1} === Train Accuracy: 0.9409 Mean Valid Accuracy: 0.9409



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9607 Valid Accuracy: 0.9599
    Fold 1: Train Accuracy: 0.9596 Valid Accuracy: 0.9586
    Fold 2: Train Accuracy: 0.9590 Valid Accuracy: 0.9598
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1} === Train Accuracy: 0.9597 Mean Valid Accuracy: 0.9595



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9649 Valid Accuracy: 0.9646
    Fold 1: Train Accuracy: 0.9657 Valid Accuracy: 0.9640
    Fold 2: Train Accuracy: 0.9661 Valid Accuracy: 0.9653
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1} === Train Accuracy: 0.9655 Mean Valid Accuracy: 0.9646



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9701 Valid Accuracy: 0.9676
    Fold 1: Train Accuracy: 0.9700 Valid Accuracy: 0.9671
    Fold 2: Train Accuracy: 0.9700 Valid Accuracy: 0.9674
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1} === Train Accuracy: 0.9700 Mean Valid Accuracy: 0.9673



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9405 Valid Accuracy: 0.9412
    Fold 1: Train Accuracy: 0.9417 Valid Accuracy: 0.9414
    Fold 2: Train Accuracy: 0.9424 Valid Accuracy: 0.9420
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05} === Train Accuracy: 0.9415 Mean Valid Accuracy: 0.9415



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9604 Valid Accuracy: 0.9590
    Fold 1: Train Accuracy: 0.9588 Valid Accuracy: 0.9595
    Fold 2: Train Accuracy: 0.9599 Valid Accuracy: 0.9589
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05} === Train Accuracy: 0.9597 Mean Valid Accuracy: 0.9591



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9653 Valid Accuracy: 0.9638
    Fold 1: Train Accuracy: 0.9663 Valid Accuracy: 0.9652
    Fold 2: Train Accuracy: 0.9646 Valid Accuracy: 0.9647
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05} === Train Accuracy: 0.9654 Mean Valid Accuracy: 0.9646



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9705 Valid Accuracy: 0.9673
    Fold 1: Train Accuracy: 0.9703 Valid Accuracy: 0.9669
    Fold 2: Train Accuracy: 0.9691 Valid Accuracy: 0.9665
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.05} === Train Accuracy: 0.9699 Mean Valid Accuracy: 0.9669



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9418 Valid Accuracy: 0.9427
    Fold 1: Train Accuracy: 0.9429 Valid Accuracy: 0.9409
    Fold 2: Train Accuracy: 0.9428 Valid Accuracy: 0.9439
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1} === Train Accuracy: 0.9425 Mean Valid Accuracy: 0.9425



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9603 Valid Accuracy: 0.9603
    Fold 1: Train Accuracy: 0.9592 Valid Accuracy: 0.9591
    Fold 2: Train Accuracy: 0.9571 Valid Accuracy: 0.9573
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1} === Train Accuracy: 0.9589 Mean Valid Accuracy: 0.9589



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9657 Valid Accuracy: 0.9647
    Fold 1: Train Accuracy: 0.9661 Valid Accuracy: 0.9652
    Fold 2: Train Accuracy: 0.9657 Valid Accuracy: 0.9642
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1} === Train Accuracy: 0.9658 Mean Valid Accuracy: 0.9647



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9692 Valid Accuracy: 0.9662
    Fold 1: Train Accuracy: 0.9706 Valid Accuracy: 0.9670
    Fold 2: Train Accuracy: 0.9698 Valid Accuracy: 0.9679
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 50, 'learning_rate': 1.1} === Train Accuracy: 0.9698 Mean Valid Accuracy: 0.9670



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9424 Valid Accuracy: 0.9431
    Fold 1: Train Accuracy: 0.9421 Valid Accuracy: 0.9422
    Fold 2: Train Accuracy: 0.9421 Valid Accuracy: 0.9409
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9} === Train Accuracy: 0.9422 Mean Valid Accuracy: 0.9421



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9627 Valid Accuracy: 0.9630
    Fold 1: Train Accuracy: 0.9630 Valid Accuracy: 0.9617
    Fold 2: Train Accuracy: 0.9631 Valid Accuracy: 0.9622
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9} === Train Accuracy: 0.9629 Mean Valid Accuracy: 0.9623



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9693 Valid Accuracy: 0.9670
    Fold 1: Train Accuracy: 0.9691 Valid Accuracy: 0.9673
    Fold 2: Train Accuracy: 0.9694 Valid Accuracy: 0.9672
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9} === Train Accuracy: 0.9693 Mean Valid Accuracy: 0.9672



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9}
    Fold 0: Train Accuracy: 0.9727 Valid Accuracy: 0.9689
    Fold 1: Train Accuracy: 0.9736 Valid Accuracy: 0.9682
    Fold 2: Train Accuracy: 0.9740 Valid Accuracy: 0.9690
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.9} === Train Accuracy: 0.9734 Mean Valid Accuracy: 0.9687



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9427 Valid Accuracy: 0.9425
    Fold 1: Train Accuracy: 0.9427 Valid Accuracy: 0.9426
    Fold 2: Train Accuracy: 0.9420 Valid Accuracy: 0.9421
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95} === Train Accuracy: 0.9425 Mean Valid Accuracy: 0.9424



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9631 Valid Accuracy: 0.9604
    Fold 1: Train Accuracy: 0.9617 Valid Accuracy: 0.9621
    Fold 2: Train Accuracy: 0.9622 Valid Accuracy: 0.9623
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95} === Train Accuracy: 0.9623 Mean Valid Accuracy: 0.9616



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9685 Valid Accuracy: 0.9661
    Fold 1: Train Accuracy: 0.9693 Valid Accuracy: 0.9677
    Fold 2: Train Accuracy: 0.9691 Valid Accuracy: 0.9667
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95} === Train Accuracy: 0.9690 Mean Valid Accuracy: 0.9668



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95}
    Fold 0: Train Accuracy: 0.9736 Valid Accuracy: 0.9680
    Fold 1: Train Accuracy: 0.9728 Valid Accuracy: 0.9682
    Fold 2: Train Accuracy: 0.9724 Valid Accuracy: 0.9685
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 0.95} === Train Accuracy: 0.9729 Mean Valid Accuracy: 0.9682



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9426 Valid Accuracy: 0.9432
    Fold 1: Train Accuracy: 0.9422 Valid Accuracy: 0.9423
    Fold 2: Train Accuracy: 0.9441 Valid Accuracy: 0.9425
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1} === Train Accuracy: 0.9430 Mean Valid Accuracy: 0.9426



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9636 Valid Accuracy: 0.9624
    Fold 1: Train Accuracy: 0.9636 Valid Accuracy: 0.9633
    Fold 2: Train Accuracy: 0.9634 Valid Accuracy: 0.9626
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1} === Train Accuracy: 0.9635 Mean Valid Accuracy: 0.9628



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9694 Valid Accuracy: 0.9680
    Fold 1: Train Accuracy: 0.9690 Valid Accuracy: 0.9669
    Fold 2: Train Accuracy: 0.9690 Valid Accuracy: 0.9671
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1} === Train Accuracy: 0.9691 Mean Valid Accuracy: 0.9673



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1}
    Fold 0: Train Accuracy: 0.9734 Valid Accuracy: 0.9687
    Fold 1: Train Accuracy: 0.9734 Valid Accuracy: 0.9683
    Fold 2: Train Accuracy: 0.9729 Valid Accuracy: 0.9677
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1} === Train Accuracy: 0.9732 Mean Valid Accuracy: 0.9682



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9424 Valid Accuracy: 0.9435
    Fold 1: Train Accuracy: 0.9439 Valid Accuracy: 0.9422
    Fold 2: Train Accuracy: 0.9425 Valid Accuracy: 0.9432
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05} === Train Accuracy: 0.9429 Mean Valid Accuracy: 0.9430



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9625 Valid Accuracy: 0.9623
    Fold 1: Train Accuracy: 0.9626 Valid Accuracy: 0.9616
    Fold 2: Train Accuracy: 0.9624 Valid Accuracy: 0.9614
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05} === Train Accuracy: 0.9625 Mean Valid Accuracy: 0.9618



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9690 Valid Accuracy: 0.9673
    Fold 1: Train Accuracy: 0.9688 Valid Accuracy: 0.9665
    Fold 2: Train Accuracy: 0.9690 Valid Accuracy: 0.9672
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05} === Train Accuracy: 0.9690 Mean Valid Accuracy: 0.9670



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05}
    Fold 0: Train Accuracy: 0.9732 Valid Accuracy: 0.9688
    Fold 1: Train Accuracy: 0.9732 Valid Accuracy: 0.9689
    Fold 2: Train Accuracy: 0.9732 Valid Accuracy: 0.9672
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.05} === Train Accuracy: 0.9732 Mean Valid Accuracy: 0.9683



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9434 Valid Accuracy: 0.9434
    Fold 1: Train Accuracy: 0.9431 Valid Accuracy: 0.9425
    Fold 2: Train Accuracy: 0.9433 Valid Accuracy: 0.9439
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1} === Train Accuracy: 0.9433 Mean Valid Accuracy: 0.9432



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9627 Valid Accuracy: 0.9624
    Fold 1: Train Accuracy: 0.9622 Valid Accuracy: 0.9635
    Fold 2: Train Accuracy: 0.9633 Valid Accuracy: 0.9609
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1} === Train Accuracy: 0.9627 Mean Valid Accuracy: 0.9623



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9685 Valid Accuracy: 0.9664
    Fold 1: Train Accuracy: 0.9688 Valid Accuracy: 0.9662
    Fold 2: Train Accuracy: 0.9687 Valid Accuracy: 0.9671
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1} === Train Accuracy: 0.9686 Mean Valid Accuracy: 0.9666



Running Cross Validation on AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1}
    Fold 0: Train Accuracy: 0.9725 Valid Accuracy: 0.9683
    Fold 1: Train Accuracy: 0.9735 Valid Accuracy: 0.9679
    Fold 2: Train Accuracy: 0.9725 Valid Accuracy: 0.9660
AdaClassifier with parameters {'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 100, 'learning_rate': 1.1} === Train Accuracy: 0.9728 Mean Valid Accuracy: 0.9674